In [14]:
import cv2
import os
import mediapipe as mp
import numpy as np
import random
import pandas as pd
import pickle
import ast
from utilities import *
from joint_angles import JointAngle, Angles
from body_parts import BodyPart
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from statsmodels.tools.eval_measures import mse
from sklearn.metrics import mean_squared_error

In [ ]:
n = 8
angles = [0] * n
bp = BodyPart()

In [ ]:
mpPose = mp.solutions.pose
video = '../assets/sample-video/sample-video05.mp4'
cap = cv2.VideoCapture(video)

In [ ]:
data = {
    'Angles': [],
    'Phase': [],
    'Score': []
}

In [ ]:
output_folder = '../assets/sample-frames/sample-video05/'
os.makedirs(output_folder, exist_ok=True)
frameCnt = 0

with mpPose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        frameName = f'frame_{frameCnt}.jpg'
        framePath = os.path.join(output_folder,frameName)
        cv2.imwrite(framePath, image)
        frameCnt += 1

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        try:
            lm = results.pose_landmarks.landmark
            jointAngle = JointAngle(lm)
            bp.cords = detect_joints(lm)
            angles = jointAngle.body_angles(bp)
            data['Angles'].append(angles)
            data['Phase'].append(None)
            data['Score'].append(None)
        except:
            pass

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)
df
df.to_csv('../dataset/angles05.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
label_encoder = LabelEncoder()

In [4]:
fileName = '../dataset/angles_final.csv'
df = pd.read_csv(fileName)

In [5]:
df['Angles'] = df['Angles'].apply(ast.literal_eval)

In [6]:
features = df['Angles'].to_list()
labels = label_encoder.fit_transform(df['Phase'])

In [7]:
len(features)

1235

In [8]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [13]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 93.52%


In [ ]:
Ypred

In [ ]:
Ytest

In [ ]:
filename = '../model/PhaseClassifier.pickle'
pickle.dump(clf, open(filename, "wb"))

In [ ]:
scores = {
    'Neck': [],
    'Left Arm': [],
    'Right Arm': [],
    'Back': [],
    'Abdomen': [],
    'Internal': [],
    'Left Leg': [],
    'Right Leg': [],
    'Angle': [],
}

In [ ]:
df = pd.DataFrame(scores)
df

In [ ]:
for i, angle in enumerate(scores):
    print(i, angle)

In [ ]:
degrees = 180

for degree in range(degrees + 1):
    score = 0
    scores['Angle'].append(degree)
    
    for i, angle in enumerate(scores):
        if i == 8:
            continue
        delta = abs(degree - angles[i])
        score = round((1 - delta / degrees) * 100)
        scores[str(angle)].append(score)    

In [ ]:
df = pd.DataFrame(scores)
df

In [ ]:
selectedAngles = df.columns.difference(['Angle'])
selectedAngles

In [ ]:
data = {
    'Angles': [],
    'Score': []
}
for i in range(0, 1000):
    score = 0
    angles = []

    for angle in selectedAngles:
        degree = random.randint(0, 180)
        score += df[angle][degree]
        angles.append(np.array(degree))
    data['Angles'].append(np.array(angles))
    data['Score'].append(np.array(round(score / 8)))

data = pd.DataFrame(data)
# data        

In [ ]:
print(data['Score'].max())
print(data['Score'].min())

In [ ]:
data.to_csv('scores.csv')

In [ ]:
for i in range(0, 1000):
    if abs(data['Angles'][i][3] - data['Angles'][i][6]) > 5:
        data['Angles'][i][3] = data['Angles'][i][6] + random.randint(-4, 4)

In [ ]:
data

In [ ]:
for i in range(len(data)):
    if data['Angles'][i][3] < 90 or data['Angles'][i][6] < 90:
        data['Score'][i] = 1
    else:
        data['Score'][i] = 0

In [ ]:
data

In [ ]:
data = {
    'Angles': [],
    'Score': []
}

angles = []
for angle in selectedAngles:
    if 1 < df[angle].min():
        index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
    elif 1 > df[angle].max():
        index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
    else:
        index = df[df[angle] == 1]['Angle'].iloc[0]
    print(index)


In [ ]:
data = {
    'Angles': [],
    'Score': []
}

for score in range(0, 101):
    angles = []
    for angle in selectedAngles:
        if score < df[angle].min():
            index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
        elif score > df[angle].max():
            index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
        else:
            index = df[df[angle] == score]['Angle'].iloc[0]
        angles.append(np.array(index))
    angles = np.array(angles)

    data['Angles'].append(angles)
    data['Score'].append(score)

data = pd.DataFrame(data)
data
        

In [ ]:
features = np.array(data['Angles'].to_list())
labels = np.array(data['Score'])

In [ ]:
labels

In [15]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

/Users/thiennguyen/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thiennguyen/miniconda3/lib/python3.10/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


KMeans(n_clusters=3, random_state=42)

In [17]:
data = df.to_dict()
data['Cluster'] = kmeans.labels_
accuracy = accuracy_score(labels, data['Cluster'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 48.10%


In [ ]:
features = data['Angles'].to_list()
labels = data['Score'].to_list()

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=1)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
Ypred

In [ ]:
np.array(Ytest)

In [25]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)
Ypred = svr.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 66.40%


In [ ]:
filename = 'scores.csv'

In [ ]:
features = df['Angle']
labels = df['Left Arm']
features = np.array(features)
labels = np.array(labels)

In [ ]:
features = features.reshape(-1, 1)

In [ ]:
features.shape

In [ ]:
features

In [ ]:
labels.shape

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn import svm
from statsmodels.tools.eval_measures import mse

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=1)

In [26]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)

In [27]:
print(reg.score(Xtrain, Ytrain))

0.8303176752309115


In [29]:
Ypred = reg.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 55.47%
